# Tractors in a Field

Looking down from above, we see a tractor plowing a field.  The field is our ASCII art canvas in some stories.  In others, it's an Argand Plane.  The tractor is on a track which puts it at every point in the field.  We model both Field and Tractor as Python classes.

If you eyeball the source code for [tractor_1.py](tractor_1.py), you will see that it uses a dict as its base data structure.  Often times we think of a rectangular matrix as more like a list of lists.  However, given dictionary keys might serve as lookup coordinates, the dict-based version may be functionally the same, and end up saving memory.

In [2]:
! python tractor_1.py

<Tractor @ (0, 0); Fuel 29; Phrase JUST USE IT>

...........
...........
...........
...........
...........
...JUST USE
 IT........
...........
...........
...........
...........


The [farmworld.py](farmworld.py) code is rather similar.  One interesting aspect of the design pattern is we have access to the tractors in the field through the field. However we also have access to the field through the tractors.  

It's as if each tractor contains a memorized version of the field, which would be true of either a human or robot driven machine.

In [3]:
import farmworld

In [4]:
farmworld._test()

make a movie
Empty field, all is peaceful

********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************

Showing the tractors in a list: 
Tractor(pos=[10, 10], facing=N, marker=O, fuel=100)
Tractor(pos=[10, 11], facing=S, marker=X, fuel=100)
===
A busy day begins...

********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
**********OX********
********************
********************
********************
********************
********************
********************
****************

In [6]:
print(dir())

['In', 'Out', '_', '_5', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_exit_code', '_i', '_i1', '_i2', '_i3', '_i4', '_i5', '_i6', '_ih', '_ii', '_iii', '_oh', 'exit', 'farmworld', 'get_ipython', 'quit', 'tractor_1']


In [11]:
thefarm = farmworld.Farm(20,20)
thefarm   # fires __repr__

Farm(20,20) @ 4457466256

In [13]:
print(str(thefarm))  # str triggers __str__ which calls Farm.render(self)

********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************



In [16]:
t1 = farmworld.Tractor(thefarm, pos=[10,10], marker="$", facing="N")
t1.plow()
print(str(thefarm))

********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
**********$*********
********************
********************
********************
********************
********************
********************
********************
********************
********************



Let's look at the Tractor type initializer, the ```__init__```:

```python
    
    class Tractor:

        def __init__(self, farm , pos = [0,0], 
                     facing="N", marker="*" , fuel=100):
            self.thefarm = farm
            self.pos = pos
            self.facing = facing
            self.marker = marker
            self.thefarm.add(self)
            self.fuel = fuel
```

The default position is ```[0,0]```, in the middle of the farm.  However no marker (character) gets planted in the farm's field, until we actually plow the ground, by activating the ```plow``` method inside of ```Tractor```.

```python

    class Tractor:
        
    ...
    
        def plow(self, marker=None):
            if marker:
                self.marker = marker
            y,x = self.pos
            self.thefarm.field[y][x] = self.marker

```

Here's where the rubber meets the road, or the tractor meets the dirt, as through the list of lists ```self.thefarm.field``` is any tractor's way to address (plow in) said field.

Notice also that our Tractor is defined as a generator, meaning it implements the ```__next__``` method, such that by triggering ```__next__``` we cause the tractor to move in whatever direction it's facing, until reaching an edge, it which point it's not smart enough to do anything put sit there, as time passes.


```python

    class Tractor:
        
        ...
        
      
        def __next__(self):
            """
            Makes me an iterator
            """
            y,x = self.pos

            if self.fuel > 0:

                if self.facing   == "N":
                    if y > 0:
                        y -= 1
                    else:
                        raise StopIteration

                elif self.facing == "S":
                    if y < self.thefarm.h - 1:
                        y += 1
                    else:
                        raise StopIteration

                elif self.facing == "W":
                    if x > 0:
                        x -= 1
                    else:
                        raise StopIteration

                elif self.facing == "E":
                    if x < self.thefarm.w - 1:
                        x += 1
                    else:
                        raise StopIteration

                self.fuel -= 1
                self.pos = (y,x)

            else:  # outta gas
                raise StopIteration

            return self.thefarm.field[y][x]
  

```


By hooking all the tractors (we may have many) to the farm's tiktok method, we cause them all to advance.

```python

    class Farm:
        
        ...
        
        def ticktock(self):  # controller
            """tick tock o' the clock
            time marches on!
            Advance each tractor in the direction it's facing,
            ignoring stuck tractors already at a fence (some
            types of Tractor are smarter than others about fences).
            """
            for tractor in self.tractors:
                try:
                    next(tractor) # state changer
                except StopIteration:
                    pass  # harmless stuck tractor signal

            self.framenumber += 1
            return self.framenumber     
```    

In [17]:
next(t1)
next(t1)
next(t1)
next(t1)
t1.plow()
print(str(thefarm))

********************
********************
********************
********************
********************
********************
**********$*********
********************
********************
********************
**********$*********
********************
********************
********************
********************
********************
********************
********************
********************
********************



In [25]:
# %load tractor_2.py
"""
CropCircleTractor

Inherits from Tractor with same __next__ based raster pattern,
however in this subclass, planting a @ occurs when the underlying
complex number in the corresponding plane does not diverge after
10 iterations of z = z * z + c.  Creates a file of ASCII art best
viewed fixed width font, small font size.

EXAMPLE OUTPUT:  https://flic.kr/p/xyNXhN

(cl) MIT License 2015 by 4dsolutions.net
"""

from tractor_1 import Tractor, Field

class CropCircleTractor(Tractor):

    def config(self, x_scale, y_scale, x_offset, y_offset):
        self.x_scale, self.y_scale = x_scale, y_scale
        self.x_offset, self.y_offset = x_offset, y_offset

    def __next__(self):
        super().__next__()  # updates pos
        c = complex((self.col + self.y_offset) * self.y_scale, 
                    (self.row + self.x_offset) * self.x_scale)
        z = complex(0,0)
        for _ in range(10):
            z = z*z + c
        if abs(z) <= 2:
            self.plant("@")
        return z
    
    def __iter__(self):
        return self

if __name__ == "__main__":
    the_field = Field(100, 250)
    the_field.add_tractor(CropCircleTractor)  # initialized as added
    the_tractor = the_field.Ts[0] # grab reference to instance
    the_tractor.config(.025, .01, -50, -200)
    the_tractor.fuel_level = 100 * 250
    for z in the_tractor:
        if the_tractor.pos == (99, 249):
            break
    with open("mandelbrot.txt", "w") as fractal:
        print(the_field, file = fractal)

In [19]:
! python tractor_2.py

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/51382948431/in/dateposted-public/" title="cropcircle_tractor_output"><img src="https://live.staticflickr.com/65535/51382948431_2743461c15.jpg" width="500" height="387" alt="cropcircle_tractor_output"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

In [24]:
# %load mandelbrot.txt

In [26]:
! python thefarm.py

movie frame 0
^....
.....
.....
.....
.....

movie frame 1
^....
.....
.....
.....
....^

movie frame 2
^....
.....
.....
....^
....^

movie frame 3
^....
.....
....^
....^
....^

movie frame 4
^....
....^
....^
....^
....^

movie frame 5
^...^
....^
....^
....^
....^

movie frame 6
^...^
....^
....^
....^
...^^

movie frame 7
^...^
....^
....^
...^^
...^^

movie frame 8
^...^
....^
...^^
...^^
...^^

movie frame 9
^...^
...^^
...^^
...^^
...^^

movie frame 10
^..^^
...^^
...^^
...^^
...^^

movie frame 11
^..^^
...^^
...^^
...^^
..^^^

movie frame 12
^..^^
...^^
...^^
..^^^
..^^^

movie frame 13
^..^^
...^^
..^^^
..^^^
..^^^

movie frame 14
^..^^
..^^^
..^^^
..^^^
..^^^

movie frame 15
^.^^^
..^^^
..^^^
..^^^
..^^^

movie frame 16
^.^^^
..^^^
..^^^
..^^^
.^^^^

movie frame 17
^.^^^
..^^^
..^^^
.^^^^
.^^^^

movie frame 18
^.^^^
..^^^
.^^^^
.^^^^
.^^^^

movie frame 19
^.^^^
.^^^^
.^^^^
.^^^^
.^^^^

Incoming updates: {'marker': '!'}
movie frame 20
^!^^^
.^^^^
.^^^^
.^^^^
.^^^^

movie fram

In [27]:
"*" * 20

'********************'

In [28]:
! python thefarm.py

movie frame 0
^....
.....
.....
.....
.....

movie frame 1
^^...
.....
.....
.....
.....

movie frame 2
^^^..
.....
.....
.....
.....

movie frame 3
^^^^.
.....
.....
.....
.....

movie frame 4
^^^^^
.....
.....
.....
.....

movie frame 5
^^^^^
^....
.....
.....
.....

movie frame 6
^^^^^
^^...
.....
.....
.....

movie frame 7
^^^^^
^^^..
.....
.....
.....

movie frame 8
^^^^^
^^^^.
.....
.....
.....

movie frame 9
^^^^^
^^^^^
.....
.....
.....

movie frame 10
^^^^^
^^^^^
^....
.....
.....

movie frame 11
^^^^^
^^^^^
^^...
.....
.....

movie frame 12
^^^^^
^^^^^
^^^..
.....
.....

movie frame 13
^^^^^
^^^^^
^^^^.
.....
.....

movie frame 14
^^^^^
^^^^^
^^^^^
.....
.....

movie frame 15
^^^^^
^^^^^
^^^^^
^....
.....

movie frame 16
^^^^^
^^^^^
^^^^^
^^...
.....

movie frame 17
^^^^^
^^^^^
^^^^^
^^^..
.....

movie frame 18
^^^^^
^^^^^
^^^^^
^^^^.
.....

movie frame 19
^^^^^
^^^^^
^^^^^
^^^^^
.....

Incoming updates: {'marker': '!'}
movie frame 20
^^^^^
^^^^^
^^^^^
^^^^^
!....

movie fram

In [29]:
import emo_tractor


💀💀💀💀💀💀💀💀💀💀
💀💀💀💀💀💀💀💀💀💀
💀💀💀💀💀💀💀💀💀💀
💀💀💀💀💀💀💀💀💀💀
💀💀💀💀👻👻👻👻👻💀
💀💀💀💀💀💀💀💀💀💀
💀💀💀💀💀💀💀💀💀💀
💀💀💀💀💀💀💀💀💀💀
💀💀💀💀💀💀💀💀💀💀
💀💀💀💀💀💀💀💀💀💀

🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙
🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙
🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙
🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙
🐙🐙🐙🐙🐅🐅🐅🐅🐅🐙
🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙
🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙
🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙
🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙
🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙
